In [4]:
using CobraTools
using JuMP
using Gurobi # use your favourite solver
using Measurements
using JLD

### Import model

In [7]:
# E. coli model
modelpath = joinpath("..", "models", "iJO1366.json") 
model = CobraTools.readmodel(modelpath)
# gibbs = CobraTools.mapGibbs(model.rxns) # very slow
gibbs = load(joinpath("..", "data", "dgzeros.jld"), "gibbs")
# ecoli_kJmol = -71.36 ± 8.55 # formation of biomass kJ/mol
# model

LoadError: MethodError: no method matching setindex!(::Measurements.Derivatives{Float64}, ::Float64, ::Tuple{Float64,Float64,UInt64})
Closest candidates are:
  setindex!(::AbstractDict, ::Any, ::Any, !Matched::Any, !Matched::Any...) at abstractdict.jl:500

In [3]:
cbmodel, v, mb, ubs, lbs = CobraTools.CBM(model);
set_optimizer(cbmodel, Gurobi.Optimizer)
set_optimizer_attribute(cbmodel, "OutputFlag", 0) # quiet

biomass_index = model[findfirst(model.rxns, "BIOMASS_Ec_iJO1366_WT_53p95M")] 
glucose_index = model[findfirst(model.rxns, "EX_glc__D_e")]
o2_index = model[findfirst(model.rxns, "EX_o2_e")]
atpm_index = model[findfirst(model.rxns, "ATPM")]

Academic license - for non-commercial use only - expires 2021-03-27


716

In [ ]:
# Fix glucose
set_normalized_rhs(lbs[glucose_index], 1.0) # lower bound
set_normalized_rhs(ubs[glucose_index], -1.0) # upper bound

# can export O2
set_normalized_rhs(lbs[o2_index], 0.0) # lower bound
set_normalized_rhs(ubs[o2_index], 1000.0) # upper bound

# remove atpm
set_normalized_rhs(lbs[atpm_index], 0.0) # lower bound
set_normalized_rhs(ubs[atpm_index], 0.0) # upper bound

@objective(cbmodel, Max, v[biomass_index])

optimize!(cbmodel) 
status = termination_status(cbmodel) == MOI.OPTIMAL

μ_max = objective_value(cbmodel)

In [ ]:
μs = range(0.0, μ_max, length=10)
ΔGs = Float64[]

for μ in μs
    set_normalized_rhs(lbs[biomass_index], μ) # lower bound
    set_normalized_rhs(ubs[biomass_index], μ) # lower bound
    @objective(cbmodel, Min, dot(v,v))
    optimize!(cbmodel)
    status = termination_status(cbmodel) == MOI.OPTIMAL
    if !status
        @warn "solve error"
    end
    fluxes = CobraTools.map_fluxes(v, model)
    \Delta blackbox(fluxes, gibbs, biomass_index, ecoli_kJmol)
    push!(ΔGs, )
end

In [ ]:
fluxes = CobraTools.map_fluxes(v, model)
blackbox(fluxes, gibbs, biomass_index, ecoli_kJmol)